In [ ]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import csv
import datetime
import random
from operator import sub

import xgboost as xgb
#from sklearn import preprocessing, ensemble
import os
#print(os.listdir("../input"))

In [ ]:
path = "./santander-product-recommendation/"
df= pd.read_csv( path + "train_ver2.csv", dtype={"sexo":str, "ind_nuevo":str, 
                                                  "ult_fec_cli_1t":str, 
                                                  "indext":str})
df.columns
df.shape

In [ ]:
limit_rows   = 7000000
df           = pd.read_csv(path + "train_ver2.csv",dtype={"sexo":str,
                                                    "ind_nuevo":str,
                                                    "ult_fec_cli_1t":str,
                                                    "indext":str}, nrows=limit_rows)
unique_ids   = pd.Series(df["ncodpers"].unique())
limit_people = 12000
unique_id    = unique_ids.sample(n=limit_people)
df           = df[df.ncodpers.isin(unique_id)]
df.describe()

In [ ]:
df["fecha_dato"] = pd.to_datetime(df["fecha_dato"],format="%Y-%m-%d")
df["fecha_alta"] = pd.to_datetime(df["fecha_alta"],format="%Y-%m-%d")
df["fecha_dato"].unique()
df["month"] = pd.DatetimeIndex(df["fecha_dato"]).month
df["age"]   = pd.to_numeric(df["age"], errors="coerce")

In [ ]:
df.isnull().any()

In [ ]:
import seaborn as sns
with sns.plotting_context("notebook",font_scale=1.5):
    sns.set_style("whitegrid")
    sns.displot(df["age"].dropna(),
#                 bins=80,
#                 kde=False,
                 color="tomato")
    plt.title("Age Distribution")
    plt.ylabel("Count")

In [ ]:
df.loc[df.age < 18,"age"]  = df.loc[(df.age >= 18) & (df.age <= 30),"age"].mean(skipna=True)
df.loc[df.age > 100,"age"] = df.loc[(df.age >= 30) & (df.age <= 100),"age"].mean(skipna=True)
df["age"].fillna(df["age"].mean(),inplace=True)
# df["age"]                  = df["age"].astype(int)

In [ ]:
df["ind_nuevo"].isnull().sum()

In [ ]:
months_active = df.loc[df["ind_nuevo"].isnull(),:].groupby("ncodpers", sort=False).size()

months_active.max()
df.loc[df["ind_nuevo"].isnull(),"ind_nuevo"] = 1

In [ ]:
df.antiguedad = pd.to_numeric(df.antiguedad,errors="coerce")
np.sum(df["antiguedad"].isnull())
df.loc[df["antiguedad"].isnull(),"ind_nuevo"].describe()
df.loc[df.antiguedad.isnull(),"antiguedad"] = df.antiguedad.min()
df.loc[df.antiguedad <0, "antiguedad"]      = 0

In [ ]:
dates=df.loc[:,"fecha_alta"].sort_values().reset_index()
median_date = int(np.median(dates.index.values))
df.loc[df.fecha_alta.isnull(),"fecha_alta"] = dates.loc[median_date,"fecha_alta"]
df["fecha_alta"].describe()

In [ ]:
pd.Series([i for i in df.indrel]).value_counts()

In [ ]:
np.sum(df["ind_actividad_cliente"].isnull())

In [ ]:
df.drop(["tipodom","cod_prov"],axis=1,inplace=True)

In [ ]:
df.isnull().any()

In [ ]:
df.loc[df.ind_actividad_cliente.isnull(),"ind_actividad_cliente"] = \
df["ind_actividad_cliente"].median()

In [ ]:
df.nomprov.unique()
df.loc[df.nomprov=="CORU\xc3\x91A, A","nomprov"] = "CORUNA, A"
df.loc[df.nomprov.isnull(),"nomprov"] = "UNKNOWN"

In [ ]:
trainData = pd.read_csv('./santander-product-recommendation/train_ver2.csv')
testData = pd.read_csv('./santander-product-recommendation/test_ver2.csv')

In [ ]:
print("testData shape : ", testData.shape)
print("trainData type : ", type(trainData))
print("trainData shape : ", trainData.shape)
print("testData type : ", type(testData))

In [ ]:
trainData

In [ ]:
testData